In [4]:
from gurobipy import *
import pandas as pd
import numpy as np

### Problem 2: Gurobi

In [5]:
# Read excel sheets and transform them into lists and matrices
file = 'OR109-2_case02_data_s3.xlsx'
demand_info=pd.read_excel(file,'Demand',index_col = 0)
initial_inventory_info=pd.read_excel(file,'Initial inventory',index_col = 0)
shipping_cost_info=pd.read_excel(file,'Shipping cost',index_col = 0)
intransit=pd.read_excel(file,'In-transit',index_col = 0)
size_info=pd.read_excel(file,'Size',index_col = 0)
price_and_cost_info=pd.read_excel(file,'Price and cost',index_col = 0)
shortage_info=pd.read_excel(file,'Shortage',index_col = 0)
vendor_product_info=pd.read_excel(file,'Vendor-Product',index_col = 0)
vendor_cost_info=pd.read_excel(file,'Vendor cost',index_col = 0)
bounds_info=pd.read_excel(file,'Bounds',index_col = 0)
conflict_info=pd.read_excel(file,'Conflict',index_col = 0)

In [6]:
demand_len = pd.read_excel(file,'Demand')
vendor_len = pd.read_excel(file, 'Vendor-Product')
conflict_info_len = pd.read_excel(file, 'Conflict')
shipping_method_len = range(3)  #SJ, constant
product_len = range(len(demand_len['Product']))  #SI
month_len = range(len(demand_len.columns) - 1)  #ST
vendor_len = range(vendor_len.max(axis=0)[1])  #SK
conflict_len = range(len(conflict_info_len['Conflict']))  #SC
print(shipping_method_len)
print(product_len)
print(month_len)
print(vendor_len)
print(conflict_len)

range(0, 3)
range(0, 500)
range(0, 26)
range(0, 9)
range(0, 25)


In [7]:
size=size_info['Size'].values.tolist()
vendor_product=vendor_product_info['Vendor'].values.tolist()
vendor_cost=vendor_cost_info['Ordering cost'].values.tolist()
bounds=bounds_info['Minimum order quantity (if an order is placed)'].values.tolist()
conflict=conflict_info.values.tolist()
#size
#bounds

In [8]:
conflict

[[40, 74],
 [298, 396],
 [238, 243],
 [374, 477],
 [208, 279],
 [28, 202],
 [353, 406],
 [49, 410],
 [4, 44],
 [244, 460],
 [197, 461],
 [251, 353],
 [432, 500],
 [190, 377],
 [338, 478],
 [136, 456],
 [256, 417],
 [126, 340],
 [348, 478],
 [164, 335],
 [77, 289],
 [63, 118],
 [357, 486],
 [293, 338],
 [252, 333]]

In [9]:
shipping_cost_info['Ocean_freight']=0
shipping_cost=shipping_cost_info.values.tolist()
shipping_cost[99][2]

0

In [10]:
backorder=shortage_info['Backorder'].values.tolist()
lost_sales=shortage_info['Lost sales'].values.tolist()
backorder_percentage=shortage_info['Backorder percentage'].values.tolist()

In [11]:
#backorder

In [12]:
#lost_sales

In [13]:
#backorder_percentage

In [14]:
pp = len(demand_len['Product'])  # int: product len
demand=demand_info.values.tolist()
initial_inventory=initial_inventory_info['Initial inventory'].values.tolist()
express_delivery_cost=shipping_cost_info['Express delivery'].values.tolist()
air_freight_cost=shipping_cost_info['Air freight'].values.tolist()
ocean_freight_cost=[0]*pp
demand[0][1]
len(initial_inventory)
pp

500

In [15]:
purchasing_cost=price_and_cost_info['Purchasing cost'].values.tolist()
holding_cost=price_and_cost_info['Holding cost'].values.tolist()
fix_cost=[100,80,50]
#purchasing_cost

In [16]:
shipping_cost=[express_delivery_cost,air_freight_cost,ocean_freight_cost]
len(shipping_cost)

3

In [17]:
initial_inventory = pd.read_excel(file, 'Initial inventory')
intransit = pd.read_excel(file, 'In-transit')
for i in range(3, pp):
    intransit.insert(loc=i+1, column = str(i+1), value=0)
in_transit=[]
for i in product_len:
    in_transit.append(list(initial_inventory.iloc[i,1:])+list(intransit.iloc[i,2:]))
#in_transit

In [18]:
cs2 = Model("cs2")

Academic license - for non-commercial use only - expires 2021-05-30
Using license file /Users/terrylu/gurobi.lic


In [19]:
#定義 y_it: =v_it, ending inventory level
y=[]
for i in product_len:
    y.append([])
    for k in month_len:
        y[i].append(cs2.addVar(vtype = GRB.INTEGER, name = "y" +  str(i+1) +'_'+ str(k+1)))

In [20]:
#定義x_ijt: product i ordered in the beginning of month t with shipping method j
x=[]
for i in product_len:
    x.append([])
    for j in shipping_method_len:
        x[i].append([])
        for t in month_len:
            x[i][j].append(cs2.addVar(lb = 0, vtype = GRB.INTEGER, name = "x"+str(i+1)+'_'+str(j+1)+'_'+str(t+1)))

In [21]:
#定義 z_jk: =y_jt, 1 if method j is used in month t
z=[]
for j in shipping_method_len:
    z.append([])
    for k in month_len:
        z[j].append(cs2.addVar(lb = 0, vtype = GRB.BINARY, name = "z" + str(j+1) +'_'+ str(k+1)))

In [22]:
#定義 w_k: =z_t, number of containers needed in month 
w=[]
for k in month_len:
    w.append(cs2.addVar(lb = 0, vtype = GRB.INTEGER , name = "w" + str(k+1)))

In [23]:
#定義 u_it: =s_it, shortage level of product i in month t
u=[]
for i in product_len:
    u.append([])
    for k in month_len:
        u[i].append(cs2.addVar(lb = 0, vtype = GRB.INTEGER, name = "u" +  str(i+1) +'_'+ str(k+1)))

In [24]:
#定義 kk_it: =k_it, 1 if the inventory level of product i at the end of month t is zero; 0 otherwise
kk=[]
for i in product_len:
    kk.append([])
    for t in month_len:
        kk[i].append(cs2.addVar(vtype = GRB.BINARY, name = "k" + str(i+1) + '_' + str(t+1)))

In [25]:
#定義d_it: demand of product i in month t as considering backorder
d=[]
for i in product_len:
    d.append([])
    for t in month_len:
        d[i].append(cs2.addVar(lb = 0, vtype = GRB.INTEGER, name = "d" + str(i+1) + '_' + str(t+1)))

In [26]:
#定義ww_kt: =w_kt, 1 if there's vendor fixed cost; 0 otherwise
ww=[]
for k in vendor_len:
    ww.append([])
    for t in month_len:
        ww[k].append(cs2.addVar(vtype = GRB.BINARY, name = "ww" + str(k+1) + '_' + str(t+1)))

In [27]:
#定義W_ik: 1 if product i is from vendor k; 0 otherwise
vendor_product_info=pd.read_excel(file,'Vendor-Product')
W=[]
for i in product_len:
    W.append([])
    for k in vendor_len:
        W[i].append(0)
for i in product_len:
    W[i][vendor_product_info.iloc[i][1]-1]=1
W[0]

[1, 0, 0, 0, 0, 0, 0, 0, 0]

In [28]:
#定義Sn^C: 1 if products are restricted; 0 otherwise (each row has two 1)
#not used
conflict_info=pd.read_excel(file, 'Conflict')
C=[]
for i in product_len:
    C.append([])
    for k in product_len:
        C[i].append(0)
for i in conflict_len:
    C[i][conflict_info.iloc[i][1]-1]=1
    C[i][conflict_info.iloc[i][2]-1]=1

In [29]:
#定義b_it: 1 if shipping method j is used; 0 otherwise
b=[]
for i in product_len:
    b.append([])
    for t in month_len:
        b[i].append(cs2.addVar(vtype = GRB.BINARY, name = "b" + str(i+1) + '_' + str(t+1)))

## Objective function

In [30]:
# setting the objective function 
cs2.setObjective(
    
    quicksum(quicksum(y[i][t]*holding_cost[i] for i in product_len) for t in month_len)+  
    # 1: holding cost
    
    quicksum(quicksum((quicksum((shipping_cost[j][i]+purchasing_cost[i])*x[i][j][t] for i in product_len) + z[j][t]*fix_cost[j]) 
                                for j in shipping_method_len) for t in month_len)+
    # 2: variable cost, purchasing cost, fixed cost
        
    quicksum(1500*w[t] for t in month_len)+ 
    # 3: container cost
    
    quicksum(quicksum(lost_sales[i]*(1-backorder_percentage[i])*(u[i][t]) for i in product_len) for t in month_len)+ 
    # 4: Si(1 − Ai)(-uit)的部分, lost sales cost
    
    quicksum(quicksum(backorder[i]*backorder_percentage[i]*(u[i][t]) for i in product_len) for t in month_len)+
    # 5: backorder cost
    
    quicksum(quicksum(ww[k][t]*vendor_cost[k] for k in vendor_len) for t in month_len)
    # 6: vendor fixed cost
    
    , GRB.MINIMIZE) 

In [31]:
# large number
M = quicksum(quicksum(demand[i][t] for t in month_len) for i in product_len)
M

<gurobi.LinExpr: 1840391.0>

In [32]:
print(type(u))
print(type(kk))

<class 'list'>
<class 'list'>


In [33]:
# Add constraints (original)
cs2.addConstrs((y[i][t] - u[i][t] == y[i][t-1] + np.array(x).sum(axis=1)[i][t] + in_transit[i][t] - d[i][t] 
               for i in product_len for t in month_len if t > 0), "1_cons")
cs2.addConstrs((y[i][t] - u[i][t] == np.array(x).sum(axis=1)[i][t] + in_transit[i][0] - d[i][t] 
               for i in product_len for t in month_len if t == 0), "1-1_cons")
cs2.addConstrs((M*kk[i][t] >= u[i][t] for i in product_len for t in month_len), "2_cons")
cs2.addConstrs((M*(1-kk[i][t]) >= y[i][t] for i in product_len for t in month_len), "3_cons")
cs2.addConstrs((M*z[j][t] >= quicksum(x[i][j][t] for i in product_len) for j in shipping_method_len for t in month_len), "4_cons")
cs2.addConstrs((w[t] >= (1/0.5)*quicksum(x[i][3-1][t]*size[i] for i in product_len) for t in month_len), "5_cons")
cs2.addConstrs((d[i][t] >= demand[i][t]+backorder_percentage[i]*u[i][t-1] 
               for i in product_len for t in month_len if t > 0), "6_cons")
cs2.addConstrs((d[i][t] >= demand[i][t]+backorder_percentage[i]*0
               for i in product_len for t in month_len if t == 0), "6-1_cons")

{(0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (1, 0): <gurobi.Constr *Awaiting Model Update*>,
 (2, 0): <gurobi.Constr *Awaiting Model Update*>,
 (3, 0): <gurobi.Constr *Awaiting Model Update*>,
 (4, 0): <gurobi.Constr *Awaiting Model Update*>,
 (5, 0): <gurobi.Constr *Awaiting Model Update*>,
 (6, 0): <gurobi.Constr *Awaiting Model Update*>,
 (7, 0): <gurobi.Constr *Awaiting Model Update*>,
 (8, 0): <gurobi.Constr *Awaiting Model Update*>,
 (9, 0): <gurobi.Constr *Awaiting Model Update*>,
 (10, 0): <gurobi.Constr *Awaiting Model Update*>,
 (11, 0): <gurobi.Constr *Awaiting Model Update*>,
 (12, 0): <gurobi.Constr *Awaiting Model Update*>,
 (13, 0): <gurobi.Constr *Awaiting Model Update*>,
 (14, 0): <gurobi.Constr *Awaiting Model Update*>,
 (15, 0): <gurobi.Constr *Awaiting Model Update*>,
 (16, 0): <gurobi.Constr *Awaiting Model Update*>,
 (17, 0): <gurobi.Constr *Awaiting Model Update*>,
 (18, 0): <gurobi.Constr *Awaiting Model Update*>,
 (19, 0): <gurobi.Constr *Awaiting Model 

In [34]:
# Add new constraints
cs2.addConstrs((M*ww[k][t] >= quicksum((W[i][k]*quicksum(x[i][j][t] for j in shipping_method_len)) for i in product_len)
              for k in vendor_len for t in month_len), "new1_cons")

#cs2.addConstrs((M*ww[k][t] >= quicksum(quicksum(W[i][k]*x[i][j][t] for j in shipping_method_len) for i in product_len)
#              for k in vendor_len for t in month_len), "new1_cons")
cs2.addConstrs((M*b[i][t] >= quicksum(x[i][j][t] for j in shipping_method_len) 
               for i in product_len for t in month_len), "new2_cons")
cs2.addConstrs((b[i][t] <= M*quicksum(x[i][j][t] for j in shipping_method_len) 
               for i in product_len for t in month_len), "new3_cons")
cs2.addConstrs((quicksum(x[i][j][t] for j in shipping_method_len) >= bounds[i]*b[i][t]
               for i in product_len for t in month_len), "new4_cons")
cs2.addConstrs((b[conflict[n][0]-1][t]+b[conflict[n][1]-1][t] <= 1
              for n in conflict_len for t in month_len), "new5_cons")

{(0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 4): <gurobi.Constr *Awaiting Model Update*>,
 (0, 5): <gurobi.Constr *Awaiting Model Update*>,
 (0, 6): <gurobi.Constr *Awaiting Model Update*>,
 (0, 7): <gurobi.Constr *Awaiting Model Update*>,
 (0, 8): <gurobi.Constr *Awaiting Model Update*>,
 (0, 9): <gurobi.Constr *Awaiting Model Update*>,
 (0, 10): <gurobi.Constr *Awaiting Model Update*>,
 (0, 11): <gurobi.Constr *Awaiting Model Update*>,
 (0, 12): <gurobi.Constr *Awaiting Model Update*>,
 (0, 13): <gurobi.Constr *Awaiting Model Update*>,
 (0, 14): <gurobi.Constr *Awaiting Model Update*>,
 (0, 15): <gurobi.Constr *Awaiting Model Update*>,
 (0, 16): <gurobi.Constr *Awaiting Model Update*>,
 (0, 17): <gurobi.Constr *Awaiting Model Update*>,
 (0, 18): <gurobi.Constr *Awaiting Model Update*>,
 (0, 19): <gurobi.Constr *Awaiting Model 

In [35]:
cs2.setParam('TimeLimit', 10*60)  # time limit
cs2.optimize()

Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 91988 rows, 104338 columns and 408308 nonzeros
Model fingerprint: 0x2f031624
Variable types: 0 continuous, 104338 integer (26312 binary)
Coefficient statistics:
  Matrix range     [1e-02, 2e+06]
  Objective range  [2e+00, 4e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+06]
Presolve removed 27004 rows and 14004 columns
Presolve time: 0.98s
Presolved: 64984 rows, 90334 columns, 335726 nonzeros
Variable types: 0 continuous, 90334 integer (25810 binary)

Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...

Concurrent spin time: 0.00s

Solved with dual simplex

Root relaxation: objective 5.742878e+08, 22381 iterations, 0.84 seconds

    Nodes    |    Current Node    |     Objective Bounds  

H 3118  3301                    5.755374e+08 5.7512e+08  0.07%   2.0  104s
H 3310  3462                    5.755370e+08 5.7512e+08  0.07%   2.0  108s
H 3471  3673                    5.755370e+08 5.7512e+08  0.07%   2.0  112s
H 3576  3673                    5.755369e+08 5.7512e+08  0.07%   2.0  112s
  3683  3674 5.7532e+08 1018 3441 5.7554e+08 5.7512e+08  0.07%   2.0  116s
  3685  3675 5.7529e+08  904 3254 5.7554e+08 5.7512e+08  0.07%   2.0  122s
  3686  3676 5.7526e+08  777 3371 5.7554e+08 5.7512e+08  0.07%   2.0  125s
  3693  3681 5.7526e+08  781 4357 5.7554e+08 5.7513e+08  0.07%   2.0  130s
  3695  3682 5.7532e+08 1010 4810 5.7554e+08 5.7515e+08  0.07%   2.0  135s
H 3698  3499                    5.754232e+08 5.7516e+08  0.05%   2.0  141s
  3704  3503 5.7519e+08  425 4915 5.7542e+08 5.7517e+08  0.04%   2.0  149s
  3706  3504 5.7533e+08 1064 5133 5.7542e+08 5.7517e+08  0.04%   2.0  150s
H 3708  3330                    5.754061e+08 5.7518e+08  0.04%   2.0  157s
  3714  3334 5.7519e+08  

In [36]:
print("z* = ", cs2.objVal)  # without new3 constraint

z* =  575352927.3749995


In [37]:
x

[[[<gurobi.Var x1_1_1 (value -0.0)>,
   <gurobi.Var x1_1_2 (value -0.0)>,
   <gurobi.Var x1_1_3 (value -0.0)>,
   <gurobi.Var x1_1_4 (value -0.0)>,
   <gurobi.Var x1_1_5 (value -0.0)>,
   <gurobi.Var x1_1_6 (value -0.0)>,
   <gurobi.Var x1_1_7 (value -0.0)>,
   <gurobi.Var x1_1_8 (value -0.0)>,
   <gurobi.Var x1_1_9 (value -0.0)>,
   <gurobi.Var x1_1_10 (value -0.0)>,
   <gurobi.Var x1_1_11 (value -0.0)>,
   <gurobi.Var x1_1_12 (value -0.0)>,
   <gurobi.Var x1_1_13 (value -0.0)>,
   <gurobi.Var x1_1_14 (value -0.0)>,
   <gurobi.Var x1_1_15 (value -0.0)>,
   <gurobi.Var x1_1_16 (value -0.0)>,
   <gurobi.Var x1_1_17 (value -0.0)>,
   <gurobi.Var x1_1_18 (value -0.0)>,
   <gurobi.Var x1_1_19 (value -0.0)>,
   <gurobi.Var x1_1_20 (value -0.0)>,
   <gurobi.Var x1_1_21 (value -0.0)>,
   <gurobi.Var x1_1_22 (value -0.0)>,
   <gurobi.Var x1_1_23 (value -0.0)>,
   <gurobi.Var x1_1_24 (value -0.0)>,
   <gurobi.Var x1_1_25 (value -0.0)>,
   <gurobi.Var x1_1_26 (value -0.0)>],
  [<gurobi.Var x1_2_